In [71]:
from dotenv import load_dotenv
import os
import praw
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn import metrics

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

import nltk
from nltk.tokenize import word_tokenize, sent_tokenize, RegexpTokenizer
from nltk.corpus import stopwords, wordnet
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [9]:
news1 = pd.read_csv('data/2024-06-27 15:03:20.894281_news.csv')
onion1 = pd.read_csv('data/2024-06-26 16:40:52.902750_onion.csv')
news2 = pd.read_csv('data/2024-06-28 15:21:52.863539_news.csv')
onion2 = pd.read_csv('data/2024-06-28 15:21:52.863539_onion.csv')
news3 = pd.read_csv('data/2024-07-02 09:23:01.740295_news.csv')
onion3 = pd.read_csv('data/2024-07-02 09:23:01.740295_onion.csv')

In [11]:
onion1.drop(columns=['Unnamed: 0'], inplace=True)

In [13]:
onion = pd.concat([onion1, onion2, onion3])

In [15]:
len(onion['created_utc'].unique())

980

In [17]:
onion.drop_duplicates('created_utc', inplace=True)

In [19]:
len(onion['created_utc'].unique())

980

In [21]:
news1.drop(columns=['Unnamed: 0'], inplace=True)

In [23]:
news = pd.concat([news1, news2, news3])

In [25]:
len(news['created_utc'].unique())

1324

In [27]:
news.drop_duplicates('created_utc', inplace=True)

In [29]:
len(news['created_utc'].unique())

1324

In [31]:
posts = pd.concat([onion, news])
posts['is_onion'] = posts['subreddit'].map({'worldnews': 0, 'TheOnion': 1})

In [33]:
posts.shape

(2304, 5)

In [38]:
posts.to_csv('data/posts.csv')

In [35]:
X = posts['title']
y = posts['is_onion']

In [43]:
1-y.mean()

0.5746527777777778

In [97]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [147]:
sw_nltk = stopwords.words('english')
sw = sw_nltk + ['thread', 'worldnews', 'live']

In [133]:
pipe_tvnb = Pipeline([
    ('tvec', TfidfVectorizer()),
    ('nb', MultinomialNB())
])

pipe_tvnb.fit(X_train, y_train)

Pipeline(steps=[('tvec', TfidfVectorizer()), ('nb', MultinomialNB())])

In [171]:
params = {
    'tvec__ngram_range': [(1, 1), (1, 2), (1, 3)],
    'tvec__stop_words': [None, 'english', sw],
    'tvec__max_features': [None, 2000, 5000, 10000, 20000]
}

gs = GridSearchCV(pipe_tvnb, param_grid=params, cv=5, n_jobs=4)

In [137]:
%%time
gs.fit(X_train, y_train)

CPU times: user 327 ms, sys: 73.5 ms, total: 401 ms
Wall time: 6.32 s


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('tvec', TfidfVectorizer()),
                                       ('nb', MultinomialNB())]),
             n_jobs=4,
             param_grid={'tvec__max_features': [None, 2000, 5000, 10000, 20000],
                         'tvec__ngram_range': [(1, 1), (1, 2), (1, 3)],
                         'tvec__stop_words': [None, 'english',
                                              ['i', 'me', 'my', 'myself', 'we',
                                               'our', 'ours', 'ourselves',
                                               'you', "you're", "you've",
                                               "you'll", "you'd", 'your',
                                               'yours', 'yourself',
                                               'yourselves', 'he', 'him', 'his',
                                               'himself', 'she', "she's", 'her',
                                               'hers', 'herself', 'it', "it's",
                                               'its', 'itself', ...]]})

In [139]:
gs.score(X_test, y_test)

0.90625

In [141]:
y_hat = gs.predict(X_test)
cm = metrics.confusion_matrix(y_test, y_hat)
cm

array([[322,  12],
       [ 42, 200]])

In [143]:
pd.DataFrame(cm, columns = ['Pred World News', 'Pred Onion'], index = ['Actual World News', 'Actual Onion'])

,Pred World News,Pred Onion
Actual World News,322,12
Actual Onion,42,200


In [188]:
pipe_cvrt = Pipeline([
    ('cvec', CountVectorizer()),
    ('rt', RandomForestClassifier(n_estimators = 300, oob_score=True))
])

In [190]:
pipe_cvrt.fit(X_train, y_train)

Pipeline(steps=[('cvec', CountVectorizer()),
                ('rt',
                 RandomForestClassifier(n_estimators=300, oob_score=True))])

In [192]:
pipe_cvrt.score(X_test, y_test)

0.8836805555555556

In [194]:
pipe_cvrt.get_params()

{'memory': None,
 'steps': [('cvec', CountVectorizer()),
  ('rt', RandomForestClassifier(n_estimators=300, oob_score=True))],
 'verbose': False,
 'cvec': CountVectorizer(),
 'rt': RandomForestClassifier(n_estimators=300, oob_score=True),
 'cvec__analyzer': 'word',
 'cvec__binary': False,
 'cvec__decode_error': 'strict',
 'cvec__dtype': numpy.int64,
 'cvec__encoding': 'utf-8',
 'cvec__input': 'content',
 'cvec__lowercase': True,
 'cvec__max_df': 1.0,
 'cvec__max_features': None,
 'cvec__min_df': 1,
 'cvec__ngram_range': (1, 1),
 'cvec__preprocessor': None,
 'cvec__stop_words': None,
 'cvec__strip_accents': None,
 'cvec__token_pattern': '(?u)\\b\\w\\w+\\b',
 'cvec__tokenizer': None,
 'cvec__vocabulary': None,
 'rt__bootstrap': True,
 'rt__ccp_alpha': 0.0,
 'rt__class_weight': None,
 'rt__criterion': 'gini',
 'rt__max_depth': None,
 'rt__max_features': 'sqrt',
 'rt__max_leaf_nodes': None,
 'rt__max_samples': None,
 'rt__min_impurity_decrease': 0.0,
 'rt__min_samples_leaf': 1,
 'rt__min_sa

In [196]:
pgrid = {
    'cvec__ngram_range': [(1, 1), (1, 2), (1, 3)],
    'cvec__stop_words': [None, 'english', sw],
    'rt__max_features': np.arange(1, 31),
    'rt__max_depth': [None, 1, 2, 3, 4, 5, 6, 7, 8],
    'rt__min_samples_leaf': np.arange(1, 31)
}

gs2 = GridSearchCV(pipe_cvrt, param_grid=pgrid, cv=5, n_jobs=2)

In [ ]:
%%time
gs2.fit(X_train, y_train)

In [89]:
rs.best_params_

{'rt__min_samples_leaf': 2, 'rt__max_features': 51, 'rt__max_depth': 9}

In [91]:
rs.score(X_test, y_test)

0.6180555555555556

In [93]:
et = ExtraTreesClassifier(n_estimators=500)
et.fit(X_train, y_train)

ValueError: could not convert string to float: 'Scientists identify new Antarctic ice sheet ‘tipping point,’ warning future sea level rise may be underestimated'